# RunnableLambda

### Run the line of code below to check the version of langchain in the current environment.
###

In [1]:
pip show langchain

Name: langchain
Version: 0.3.20
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: c:\users\hp\anaconda3\envs\longchain_envllm\lib\site-packages
Requires: async-timeout, langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_core.runnables import RunnableLambda

In [3]:
find_sum = lambda x: sum(x)

In [4]:
find_sum([1, 2, 5])

8

In [5]:
find_square = lambda x: x**2

In [6]:
find_square(8)

64

In [7]:
runnable_sum = RunnableLambda(lambda x: sum(x))

In [8]:
runnable_sum.invoke([1, 2, 5])

8

In [9]:
runnable_square = RunnableLambda(lambda x: x**2)

In [10]:
runnable_square.invoke(8)

64

In [16]:
chain = runnable_sum | runnable_square

In [17]:
chain.invoke([1, 2, 5])

64

In [18]:
chain.get_graph().print_ascii()

+-------------+  
| LambdaInput |  
+-------------+  
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
+--------------+ 
| LambdaOutput | 
+--------------+ 


# The @chain Decorator

In [19]:
from langchain_core.runnables import chain

In [20]:
def find_sum(x):
    return sum(x)

def find_square(x):
    return x**2

In [21]:
chain1 = RunnableLambda(find_sum) | RunnableLambda(find_square)

In [22]:
chain1.invoke([1, 2, 5])

64

#### Decorators are functions that modify the behavior of another function or a class, enhancing their functionality.
####
#### Here, the chain decorator wraps the runnable some function in a runnable lambda, just as we manually did above.

In [23]:
@chain
def runnable_sum(x):
    return sum(x)

@chain
def runnable_square(x):
    return x**2

In [24]:
type(runnable_sum), type(runnable_square)

(langchain_core.runnables.base.RunnableLambda,
 langchain_core.runnables.base.RunnableLambda)

In [25]:
chain2 = runnable_sum | runnable_square

In [26]:
chain2.invoke([1, 2, 5])

64

# Adding Memory to a Chain (Part 1): Implementing the Setup

In [27]:
from langchain.memory import ConversationSummaryMemory
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [28]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{message_log}

Human: 
{question}

AI:
'''


In [29]:
prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [30]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

C:\Users\HP\anaconda3\envs\longchain_envllm\lib\site-packages\IPython\core\interactiveshell.py:3519: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [31]:
chain = prompt_template | chat | StrOutputParser()

In [32]:
chain.invoke({'message_log':'''The AI provides an interesting fact about octopuses, stating that they have three hearts. 
Two hearts pump blood to the gills, while the third heart pumps it to the rest of the body.''', 
              'question':"Can you elaborate a bit more on this fact?"})

"Absolutely! Octopuses are fascinating creatures with a unique cardiovascular system. They have three hearts, each serving a specific purpose. Two of these hearts, known as branchial hearts, are located near each of the octopus's two gills. These branchial hearts pump blood to the gills, where it gets oxygenated.\n\nThe third heart, known as the systemic heart, pumps the oxygenated blood from the gills to the rest of the body. This heart actually stops beating when the oct"

In [33]:
chat_memory = ConversationSummaryMemory(llm = ChatOpenAI(), memory_key = 'message_log')

C:\Users\HP\AppData\Local\Temp\ipykernel_13360\3865083148.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory = ConversationSummaryMemory(llm = ChatOpenAI(), memory_key = 'message_log')


In [34]:
chat_memory.load_memory_variables({})

{'message_log': ''}

# RunnablePassthrough with Additional Keys

In [35]:
from langchain_core.runnables import RunnablePassthrough

In [36]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{message_log}

Human: 
{question}

AI:
'''

prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [37]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

C:\Users\HP\anaconda3\envs\longchain_envllm\lib\site-packages\IPython\core\interactiveshell.py:3519: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [38]:
chain = prompt_template | chat | StrOutputParser()

In [39]:
chat_memory = ConversationSummaryMemory(llm = ChatOpenAI(), memory_key = 'message_log')

In [40]:
chat_memory.load_memory_variables({})

{'message_log': ''}

In [41]:
RunnablePassthrough.assign(message_log = chat_memory.load_memory_variables).invoke(
    {'question':"Can you give me an interesting fact I probably didn't know about?"})

{'question': "Can you give me an interesting fact I probably didn't know about?",
 'message_log': {'message_log': ''}}

In [42]:
RunnablePassthrough.assign(first_letter = lambda x: list(x['input'])[0], 
                           second_letter = lambda x: list(x['input'])[1]
                          ).invoke({'input':'hi'})

{'input': 'hi', 'first_letter': 'h', 'second_letter': 'i'}